# Recap

In order of priority/time taken

1. pandas init dict
    - `basal_area_aw_df = pd.DataFrame(columns=['BA_Aw'], index=xrange(max_age))`
    - find a faster way to create this data frame
    - relax the tolerance for aspen
2. pandas set item
    - use at method 
    - http://pandas.pydata.org/pandas-docs/stable/indexing.html#fast-scalar-value-getting-and-setting
3. lambdas
    - use cython for the gross tot vol and merch vol functions
    - might be wise to refactor these first to have conventional names, keyword arguments, and a base implementation to get rid of the boilerplate
    - don't be deceived - the callable is a miniscule portion; series.__getitem__ is taking most of the time
    - again, using .at here would probably be a significant improvement
4. basalareaincremementnonspatialaw
    - this is actually slow because of the number of times the BAFromZeroToDataAw function is called as shown above
    - relaxing the tolerance may help
    - indeed the tolerance is 0.01 * some value while the other factor finder functions have 0.1 tolerance i think
    - can also use cython for the increment functions

do a profiling run with IO (of reading input data and writing the plot curves to files) in next run


# Characterize what is happening

Indexing with df[] or series[] is slow for scalars (lambdas, pandas set)
basalareaincrement is running a lot for aw, use the same tolerance as is used for other species

merchvol, increment, and gross vol functions use pure python. cython would be effective.

# Decide on the action

- use same tolerance for aw as other species
- use at instead of [] or ix? - compare these in MWE
- creating data frame is slow, maybe because its fromdict. see if this can be improved

# MWEs

In [6]:
import pandas as pd
import numpy as np

## init from dict and xrange index vs from somethign else

### Timings

In [8]:
%%timeit
d = pd.DataFrame(columns=['A'], index=xrange(1000))

The slowest run took 38.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 368 µs per loop


In [9]:
%%timeit
d = pd.DataFrame(columns=['A'], index=xrange(1000), dtype='float')

The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 418 µs per loop


In [13]:
%%timeit
d = pd.DataFrame({'A': np.zeros(1000)})

The slowest run took 5.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 275 µs per loop


The problem here is that dataframe init being called 7000 times because of the aw ba factor finder

Maybe it's not worth using a data frame here. use a list or numpy and then convert to dataframe when the factor is found, e.g.:

In [14]:
%%timeit
for _ in xrange(5000):
    d = pd.DataFrame(columns=['A'], index=xrange(1000))

1 loop, best of 3: 2.13 s per loop


In [17]:
%%timeit
for _ in xrange(5000):
    d = np.zeros(1000)

100 loops, best of 3: 14.1 ms per loop


### Review the code to see how this can be applied

The numpy/purepython approach as potential

But there's a couple issues for which the code must be examined

The problem comes from the following call chain

`simulate_forwards_df` (called 1x) ->  
`get_factors_for_all_species` (called 10x, 1x per plot) ->  
`BAfactorFinder_Aw` (called 2x, 1x per plot that has aw) ->  
`BAfromZeroToDataAw` (called 7191 times, most of which in this chain) ->   
`DataFrame.__init__` (called 7932 times, most of which in this chain) ...  

why does `BAfromZeroToDataAw` create a dataframe? It's good to see the code:

First, `simulate_forwards_df` calls `get_factors_for_all_species` and then `BAfromZeroToDataAw` with some parameters and simulation choice of false

Note that when `simulation==False`, that is the only time that the list is created. otherwise the list is left empty.

Note also that `simulation_choice` defaults to `True` in forward simulation, i.e. for when `BAfromZeroToData__` are called from forward simulation.

`get_factors_for_all_species` calls factor finder functions for each species, if the species is present, and returns a dict of the factors

`BAfactorFinder_Aw` is the main suspect, for sime reason aspen has a harder time converging, so the loop in this function runs many times


It calls `BAfromZeroToDataAw` with `simulation_choice` of `'yes'` and `simulation=True` **BUT IT ONLY USES THE 1ST RETURN VALUE**

## slow lambdas

**below is left here for the record, but the time is actually spent in getitem, not so much in the callables applied, that is an easy fix**

With the df init improved by using np array, the next suspect is the lambdas. The method for optimizing is generally to use cython, the functiosn themselves can be examined for opportunities:

they are pretty basic - everything is a float.

``` python
def MerchantableVolumeAw(N_bh_Aw, BA_Aw, topHeight_Aw, StumpDOB_Aw,
                         StumpHeight_Aw, TopDib_Aw, Tvol_Aw):
    # ...
    if N_bh_Aw > 0:
        k_Aw = (BA_Aw * 10000.0 / N_bh_Aw)**0.5
    else:
        k_Aw = 0

    if k_Aw > 0 and topHeight_Aw > 0:
        b0 = 0.993673
        b1 = 923.5825
        b2 = -3.96171
        b3 = 3.366144
        b4 = 0.316236
        b5 = 0.968953
        b6 = -1.61247
        k1 = Tvol_Aw * (k_Aw**b0)
        k2 = (b1* (topHeight_Aw**b2) * (StumpDOB_Aw**b3) * (StumpHeight_Aw**b4) * (TopDib_Aw**b5)  * (k_Aw**b6)) + k_Aw
        MVol_Aw = k1/k2
    else:
        MVol_Aw = 0

    return MVol_Aw

```

``` python
def GrossTotalVolume_Aw(BA_Aw, topHeight_Aw):
    # ...
    Tvol_Aw = 0

    if topHeight_Aw > 0:
        a1 = 0.248718
        a2 = 0.98568
        a3 = 0.857278
        a4 = -24.9961
        Tvol_Aw = a1 * (BA_Aw**a2) * (topHeight_Aw**a3) * numpy.exp(1+(a4/((topHeight_Aw**2)+1)))

    return Tvol_Aw
```

### Timings for getitem

There's a few ways to get an item from a series:

In [23]:
d = pd.Series(np.random.randint(0,100, size=(100)), index=['%d' %d for d in xrange(100)])

In [25]:
%%timeit
d['1']

The slowest run took 14.05 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.4 µs per loop


In [26]:
%%timeit 
d.at('1')

The slowest run took 37.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.42 µs per loop


In [27]:
%%timeit
d.loc('1')

The slowest run took 33.90 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 1.41 µs per loop


`loc` or `at` are faster than `[]` indexing.

# Revise the code

Go on. Do it.

# Review code changes

In [58]:
%%bash
git log --since 2016-11-09 --oneline

4c978af Avoid call pandas `.at` 750 times for each plot
6b541d5 Finish optimization iteration 2
f4cdfb2 Add profile after indexing and init optimizations
2d84c77 Use a generic function for merch and gross vol
9528062 Index with .at[] instead of [], remove repetition
2de01cc Record time reduction for df init
900e847 Add timing for old gypsy simulate
cf7d73a Add timing
3f4fab3 Upd notebook - remove functions since code was upd
f72349d Revise test data and tolerance
4c5d608 Revise test data
4091820 Revise tests to use np.testing.assert_allclose
da080a7 Remove simulation from factorfinder functions
6a9fcd2 Remove redundant if statements
0339feb Factor data frames out of the fromzerotodata funcs
6e68adc cleanup style
179440f Add ian as author
299a60c Add plan to reduce df.__init__ calls
c9f0c00 Explain forward simulation loop vs fromzerotodata
fa2302d cleanup redundant files
4b46fe0 add notebook for next iteration of optimization
d107dc4 Finish performance analysis after append refactor


In [59]:
! git diff HEAD~23 ../gypsy

diff --git a/gypsy/GYPSYNonSpatial.py b/gypsy/GYPSYNonSpatial.py
index 24ec050..7b93c0f 100644
--- a/gypsy/GYPSYNonSpatial.py
+++ b/gypsy/GYPSYNonSpatial.py
@@ -15,6 +15,7 @@ N_bh_Aw = estimated N and should be equal N_Aw (for Aspen in this case Aw)
 import os
 import logging
 import numpy
+import numpy as np
 import pandas as pd
 import matplotlib.pyplot as plt
 
@@ -878,11 +879,7 @@ def densities_and_SCs_to_250(**kwargs):
             topHeight_Pl = ComputeGypsyTreeHeightGivenSiteIndexAndTotalAge('Pl', SI_bh_Pl, tage_Pl)
 
 
-        SC_F = SCestimate(N_bh_AwT, N_bh_SbT, N_bh_SwT, N_bh_PlT)
-        SC_Aw = SC_F[0]
-        SC_Sw = SC_F[1]
-        SC_Sb = SC_F[2]
-        SC_Pl = SC_F[3]
+        SC_Aw, SC_Sw, SC_Sb, SC_Pl = SCestimate(N_bh_AwT, N_bh_SbT, N_bh_SwT, N_bh_PlT)
 
         densities_along_time.append({'N_bh_AwT': N_bh_AwT, 'N_bh_SwT': N_bh_SwT, 'N_bh_SbT': N_bh_SbT, 'N_bh_PlT': N_bh_PlT,
                                      'SC_Aw': SC_Aw, 'SC_Sw': SC_Sw, 'SC_Sb':SC_Sb

# Tests

Do tests still pass?

# Run timings

In [3]:
%%bash
# git checkout dev
# time gypsy simulate ../private-data/prepped_random_sample_300.csv --output-dir tmp
# rm -rfd tmp

# real	8m18.753s
# user	8m8.980s
# sys	0m1.620s

In [22]:
%%bash
# after factoring dataframe out of zerotodata functions
# git checkout -b da080a79200f50d2dda7942c838b7f3cad845280 df-factored-out-zerotodata
# time gypsy simulate ../private-data/prepped_random_sample_300.csv --output-dir tmp
# rm -rfd tmp

# real	5m51.028s
# user	5m40.130s
# sys	0m1.680s

Removing the data frame init gets a 25% time reduction

In [41]:
%%bash
# after using a faster indexing method for the arguments put into the apply functions
# git checkout 6b541d5fb8534d6fb055961a9d5b09e1946f0b46 -b applys-use-faster-getitem
# time gypsy simulate ../private-data/prepped_random_sample_300.csv --output-dir tmp
# rm -rfd tmp

# real	6m16.021s
# user	5m59.620s
# sys	0m2.030s

Hm, this actually got worse, although it is a small sample......... if anything i suspect its because we're calling row.at[] instead of assigning the variable outside the loop. It's ok as the code has less reptition, it's a good tradeoff.

In [55]:
%%bash
# after fixing `.at` redundancy - calling it in each apply call
# git checkout 4c978aff110001efdc917ed60cb611139e1b54c9 -b remove-getitem-redundancy
# time gypsy simulate ../private-data/prepped_random_sample_300.csv --output-dir tmp
# rm -rfd tmp

# real	5m36.407s
# user	5m25.740s
# sys	0m2.140s

It doesn't totrally remove redundancy, we still get an attr/value of an object, but now its a dict instead of a pandas series. Hopefully it's faster. Should have tested first using MWE.

It is moderately faster. Not much.

Leave cython optimization for next iteration

# Run profiling

In [42]:
from gypsy.forward_simulation import simulate_forwards_df

In [44]:
data = pd.read_csv('../private-data/prepped_random_sample_300.csv', index_col=0, nrows=10)

In [54]:
%%prun -D forward-sim-2.prof -T forward-sim-2.txt -q
result = simulate_forwards_df(data)

 
*** Profile stats marshalled to file u'forward-sim-2.prof'. 

*** Profile printout saved to text file u'forward-sim-2.txt'. 


In [56]:
!head forward-sim-2.txt

         4167146 function calls (4034230 primitive calls) in 36.370 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   492069    7.050    0.000    7.050    0.000 GYPSYNonSpatial.py:428(BasalAreaIncrementNonSpatialAw)
     7191    2.481    0.000    9.716    0.001 GYPSYNonSpatial.py:956(BAfromZeroToDataAw)
    90290    2.419    0.000    8.879    0.000 base.py:2116(get_value)
    90280    1.686    0.000   17.526    0.000 indexing.py:1654(__getitem__)
   369310    1.517    0.000    2.398    0.000 {isinstance}


# Compare performance visualizations

Now use either of these commands to visualize the profiling

```
pyprof2calltree -k -i forward-sim-1.prof forward-sim-1.txt

# or

dc run --service-ports snakeviz notebooks/forward-sim-1.prof
```

### Old

![definitive reference profile screenshot](forward-sim-1-performance-icicle.png)

### New

![2nd iteration performance](forward-sim-2a-performance.png)

## Summary of performance improvements

forward_simulation is now 2x faster than last iteration, 8 times in total, due to the changes outlined in the code review section above

on my hardware, this takes 1000 plots to ~4 minutes

on carol's hardware, this takes 1000 plots to ~13 minutes

For 1 million plots, we're looking at 2 to 9 days on desktop hardware



# Profile with I/O


In [ ]:
! rm -rfd gypsy-output

In [ ]:
output_dir = 'gypsy-output'

In [20]:
%%prun -D forward-sim-2.prof -T forward-sim-2.txt -q
# restart the kernel first
data = pd.read_csv('../private-data/prepped_random_sample_300.csv', index_col=0, nrows=10)
result = simulate_forwards_df(data)
os.makedirs(output_dir)
for plot_id, df in result.items():
    filename = '%s.csv' % plot_id
    output_path = os.path.join(output_dir, filename)
    df.to_csv(output_path)


 
*** Profile stats marshalled to file u'forward-sim-1.prof'. 

*** Profile printout saved to text file u'forward-sim-1.txt'. 


# Identify new areas to optimize



- from last time:
    - parallel (3 cores) gets us to 2 - 6 days - save for last
    - AWS with 36 cores gets us to 4 - 12 hours ($6.70 - $20.10 USD on a c4.8xlarge instance in US West Region)
    - aws lambda and split up the data 
- now:
    - getting items in apply is still slow - vectorize the functions
    - cython for icnrement functions epsecially bA